In [1]:
# Install necessary packages
%pip install nltk pandas matplotlib seaborn numpy scikit-learn joblib


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import re
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
import numpy as np

nltk.download('vader_lexicon')
nltk.download('punkt_tab')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [3]:
requests = pd.read_csv('/workspaces/IST707.Pothole-Prediction-SYR/data/raw/SYRCityline_Requests_(2021-Present).csv')
weather = pd.read_csv('/workspaces/IST707.Pothole-Prediction-SYR/data/raw/40_Year_Temperature_Dataset_SU-2.csv')
ratings_2021 = pd.read_csv('/workspaces/IST707.Pothole-Prediction-SYR/data/raw/Pavement_Ratings_(2021)-2.csv')
ratings_2022 = pd.read_csv('/workspaces/IST707.Pothole-Prediction-SYR/data/raw/Syracuse_Pavement_Ratings_(2022).csv')
ratings_2023 = pd.read_csv('/workspaces/IST707.Pothole-Prediction-SYR/data/raw/Syracuse_Ratings_Open_Data_Copy_2023_8469753568545483898.csv')
streets = pd.read_csv('/workspaces/IST707.Pothole-Prediction-SYR/data/raw/City_Streets_2011.csv')

Approach:
1) Load the request data, filter out the pothole requests from 2021-2023
2) Standardize the address in potholes and match with the street dictionary 
3) Concatenate the rating data
4) Filter out temperature data
5) Merge  

In [4]:
requests['Description'] = requests['Description'].astype(str)
potholes = requests[requests['Category'] == 'Potholes']
# Ensure the 'Date' column is in datetime format
potholes = potholes[['Rating', 'Address', 'Description', 'Created_at_local', 'Minutes_to_closed']]
potholes['Date'] = pd.to_datetime(potholes['Created_at_local'], format='%m/%d/%Y - %I:%M%p').dt.date
# Define the date range
start_date = pd.to_datetime("2021-01-01").date()
end_date = pd.to_datetime("2023-12-31").date()
# Filter the DataFrame to the specified range
filtered_potholes = potholes[(potholes['Date'] >= start_date) & (potholes['Date'] <= end_date)]

Clean up the street names column

In [5]:
# Function to extract cleaner street names
def extract_clean_street_name(address):
    # More strict extraction to focus on just the name and type
    match = re.search(r'\b([A-Za-z]+)\s*(St|Street|Ave|Avenue|Blvd|Boulevard|Dr|Drive|Rd|Road|Ln|Lane|Way|Circle|Cir)\b', address, re.IGNORECASE)
    if match:
        # Normalize the street name by removing common suffixes
        street_name = match.group(1).strip().upper()  # Extract and normalize the base name
        return street_name
    else:
        return None

# Apply the revised function to the Address column
filtered_potholes['Street'] = filtered_potholes['Address'].apply(extract_clean_street_name)
filtered_potholes['Street'] = filtered_potholes['Street'].astype(str)

/tmp/ipykernel_5171/3488095503.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_potholes['Street'] = filtered_potholes['Address'].apply(extract_clean_street_name)
/tmp/ipykernel_5171/3488095503.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_potholes['Street'] = filtered_potholes['Street'].astype(str)


Caclulate sentiment analysis of potholes in relation to the street names

In [6]:
sia = SentimentIntensityAnalyzer()

def sentiment_score(description):
    # Tokenize the description
    tokens = word_tokenize(description)
    
    # Join the tokens back into a string (optional step, depending on your approach)
    text = ' '.join(tokens)
    
    # Calculate sentiment score
    score = sia.polarity_scores(text)['compound']
    
    # Return the compound sentiment score
    return score

filtered_potholes['SentimentScore'] = filtered_potholes['Description'].apply(sentiment_score)

def categorize_severity(score):
    if score <= -0.5:
        return 'high'
    elif score <= 0:
        return 'medium'
    else:
        return 'low'

filtered_potholes['SeverityLevel'] = filtered_potholes['SentimentScore'].apply(categorize_severity)

potholes_agg = filtered_potholes.groupby(['Date', 'Street']).agg({
    'SentimentScore': 'mean',  # Average sentiment score per day
    'SeverityLevel': lambda x: x.mode()[0] if not x.empty else None,  # Most common severity level per day
    'Description': 'count',  # Count the number of reports per day
    'Rating': 'mean',  # Average rating
    'Minutes_to_closed': 'mean'  # Average minutes to close
}).rename(columns={'Description': 'Potholes_Count'}).reset_index()

# potholes_agg['Date'] = pd.to_datetime(potholes_agg['Date'])
potholes_agg.head()

/tmp/ipykernel_5171/4084124416.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_potholes['SentimentScore'] = filtered_potholes['Description'].apply(sentiment_score)
/tmp/ipykernel_5171/4084124416.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_potholes['SeverityLevel'] = filtered_potholes['SentimentScore'].apply(categorize_severity)


,Date,Street,SentimentScore,SeverityLevel,Potholes_Count,Rating,Minutes_to_closed
0,2021-06-15,MCBRIDE,-0.8054,high,2,2.0,32692.0
1,2021-06-15,None,0.0000,medium,1,2.0,32918.0
2,2021-06-15,SOLAR,0.0000,medium,1,2.0,10439.0
3,2021-06-22,CARBON,0.0000,medium,1,2.0,32915.0
4,2021-06-22,COLVIN,0.0000,medium,1,2.0,31500.0


In [7]:
ratings_2021['Year'] = 2021
ratings_2022['Year'] = 2022
ratings_2023['Year'] = 2023

ratings_2021.columns = ratings_2022.columns = ratings_2023.columns
# Concatenate the data
combined_data = pd.concat([ratings_2021, ratings_2022, ratings_2023], ignore_index=True)
combined_data = combined_data.rename(columns={'Rating2023': 'Pavement_rating'})

agg_rating = combined_data.groupby(['Year', 'STREET_NAM']).agg({
    'Pavement_rating': 'mean',  # Assuming you want the mean rating; adjust the aggregation as needed
    'Miles': 'sum',  # Total miles
    'Shape__Length': 'sum'  # Total shape length
}).reset_index()

Extract the street names and aggregate ratings, miles and shapes to each street

In [8]:
# Extract unique street names from the 'NAME' column
streets_unique = streets['NAME'].unique()
streets_unique

agg_rating['STREET_NAM'] = agg_rating['STREET_NAM'].str.upper()

# Function to find a partial match in streets_unique
def find_partial_match(street_name):
    for unique_name in streets_unique:
        if unique_name in street_name:
            return unique_name
    return np.nan  # or return street_name if you want to keep original for no matches

# Apply the matching function
agg_rating['Matched_Street'] = agg_rating['STREET_NAM'].apply(find_partial_match)

aggregated_rating_st = agg_rating.groupby(['Matched_Street', 'Year']).agg({
    'Pavement_rating': 'mean',  # Average rating per matched street
    'Miles': 'sum',  # Total miles per matched street
    'Shape__Length': 'sum'  # Total shape length
}).reset_index()

More cleanup and aggregation of the aggregated data set based on street names

In [9]:
potholes_agg['Year'] = pd.to_datetime(potholes_agg['Date']).dt.year.astype('int64')

# Function to normalize street names (remove common suffixes and convert to uppercase)
def normalize_street_name(street_name):
    suffixes = r'(STREET|ST|AVENUE|AVE|BOULEVARD|BLVD|DRIVE|DR|ROAD|RD|LANE|LN|WAY|CIRCLE|CIR)$'
    return pd.Series(street_name).replace(suffixes, '', regex=True).str.strip().str.upper()

# Normalize street names in both dataframes
aggregated_rating_st['Normalized_Street'] = normalize_street_name(aggregated_rating_st['Matched_Street'])
potholes_agg['Normalized_Street'] = normalize_street_name(potholes_agg['Street'])

# Perform the join based on the year
merged_df = pd.merge(aggregated_rating_st, potholes_agg, left_on='Year', right_on='Year', how='left')

# Define a function to check for partial matches
def is_partial_match(street_name_agg, street_name_potholes):
    if pd.isna(street_name_agg) or pd.isna(street_name_potholes):
        return False
    return street_name_agg in street_name_potholes or street_name_potholes in street_name_agg

# Filter the merged dataframe for partial matches of street names
filtered_merged_df = merged_df[merged_df.apply(lambda x: is_partial_match(x['Normalized_Street_x'], x['Normalized_Street_y']), axis=1)]

# Drop the columns used for matching
filtered_merged_df.drop(columns=['Normalized_Street_x', 'Normalized_Street_y'], inplace=True)

/tmp/ipykernel_5171/3595299375.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_merged_df.drop(columns=['Normalized_Street_x', 'Normalized_Street_y'], inplace=True)


Complete the date cleanup on the new weather data frame

In [10]:
# Convert 'Year', 'Month', and 'Day' columns to a single datetime column
weather['Date'] = pd.to_datetime(weather[['Year', 'Month', 'Day']]).dt.date
weather = weather[weather['Year'] >= 2021]
weather = weather.drop(['Year', 'Month', 'Day'], axis=1)

Merge the weather and existing aggregated data frame on dates, and then drop the year and street columns

In [11]:
merged_data = pd.merge(weather, filtered_merged_df, on='Date', how='left')

In [12]:
merged_data = merged_data.drop(['Year', 'Street'], axis=1)

In [13]:
merged_data.to_csv('/workspaces/IST707.Pothole-Prediction-SYR/data/processed/final_data.csv')